Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [0]:
# pd.options.display.float_format = '{:,}'.format

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Read New York City property sales data
import pandas as pd
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

Your code starts here:

In [267]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,nan,R4,"447 WEST 18TH STREET, PH12A",PH12A,"10,011.0",1.0,0.0,1.0,"10,733","1,979.0","2,007.0",2,R4,$ - 0,01/01/2019
1,1,FASHION,21 OFFICE BUILDINGS,4,812,68,nan,O5,144 WEST 37TH STREET,NaN,"10,018.0",0.0,6.0,6.0,"2,962","15,435.0","1,920.0",4,O5,$ - 0,01/01/2019
2,1,FASHION,21 OFFICE BUILDINGS,4,839,69,nan,O5,40 WEST 38TH STREET,NaN,"10,018.0",0.0,7.0,7.0,"2,074","11,332.0","1,930.0",4,O5,$ - 0,01/01/2019
3,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,nan,R4,"1 SHERIDAN SQUARE, 8C",8C,"10,014.0",1.0,0.0,1.0,0,500.0,0.0,2,R4,$ - 0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,nan,R1,"20 EAST 65TH STREET, B",B,"10,065.0",1.0,0.0,1.0,0,"6,406.0",0.0,2,R1,$ - 0,01/01/2019


In [268]:
# Change column names: replace spaces with underscores
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()
df.columns

Index(['borough', 'neighborhood', 'building_class_category',
       'tax_class_at_present', 'block', 'lot', 'ease-ment',
       'building_class_at_present', 'address', 'apartment_number', 'zip_code',
       'residential_units', 'commercial_units', 'total_units',
       'land_square_feet', 'gross_square_feet', 'year_built',
       'tax_class_at_time_of_sale', 'building_class_at_time_of_sale',
       'sale_price', 'sale_date'],
      dtype='object')

In [0]:
# Get Pandas Profiling Report
from pandas_profiling import ProfileReport
# ProfileReport(df,minimal=True)

In [270]:
# Keep just the subset of data for the Tribeca neighborhood
# Check how many rows you have now. (Should go down from > 20k rows to 146)
tribeca = df[df['neighborhood'] == 'TRIBECA']
tribeca.shape

(146, 21)

In [271]:
# Q. What's the date range of these property sales in Tribeca?
print(tribeca['sale_date'].max())
print(tribeca['sale_date'].min())

04/30/2019
01/03/2019


In [0]:
# The Pandas Profiling Report showed that SALE_PRICE was read as strings
# Convert it to integers
df['sale_price'] = df['sale_price'].str.strip('$').str.strip(' ').str.strip('- ')

In [273]:
df['sale_price'][40]

'0'

In [0]:
df['sale_price'] = df['sale_price'].str.replace(',', '')

In [0]:
df['sale_price'] = df['sale_price'].astype(int)

In [276]:
# Q. What is the maximum SALE_PRICE in this dataset?
df['sale_price'].max()

850000000

In [277]:
df[df['sale_price'] == df['sale_price'].max()]

,borough,neighborhood,building_class_category,tax_class_at_present,block,lot,ease-ment,building_class_at_present,address,apartment_number,zip_code,residential_units,commercial_units,total_units,land_square_feet,gross_square_feet,year_built,tax_class_at_time_of_sale,building_class_at_time_of_sale,sale_price,sale_date
7745,1,FASHION,22 STORE BUILDINGS,4,840,42,nan,K3,424 5 AVENUE,NaN,"10,018.0",0.0,4.0,4.0,"51,271","697,029.0","1,920.0",4,K3,850000000,02/08/2019


In [278]:
# Get value counts of TOTAL_UNITS
# Q. How many property sales were for multiple units?
multi_units = df[df['total_units'] > 1]
multi_units['total_units'].sum()

43267.0

In [279]:
# Keep only the single units
single_units = df[df['total_units'] == 1]
single_units.shape

(10204, 21)

In [280]:
# Q. Now what is the max sales price? How many square feet does it have?
top_single_unit = single_units[single_units['sale_price'] == single_units['sale_price'].max()]
top_single_unit['gross_square_feet']

3995   23,029.0
Name: gross_square_feet, dtype: float64

In [281]:
# Q. How often did $0 sales occur in this subset of the data?

# There's a glossary here: 
# https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page

# It says:
# A $0 sale indicates that there was a transfer of ownership without a 
# cash consideration. There can be a number of reasons for a $0 sale including 
# transfers of ownership from parents to children. 
zero_price = single_units[single_units['sale_price'] == 0]
zero_price.head()

,borough,neighborhood,building_class_category,tax_class_at_present,block,lot,ease-ment,building_class_at_present,address,apartment_number,zip_code,residential_units,commercial_units,total_units,land_square_feet,gross_square_feet,year_built,tax_class_at_time_of_sale,building_class_at_time_of_sale,sale_price,sale_date
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,nan,R4,"447 WEST 18TH STREET, PH12A",PH12A,"10,011.0",1.0,0.0,1.0,"10,733","1,979.0","2,007.0",2,R4,0,01/01/2019
3,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,nan,R4,"1 SHERIDAN SQUARE, 8C",8C,"10,014.0",1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,nan,R1,"20 EAST 65TH STREET, B",B,"10,065.0",1.0,0.0,1.0,0,"6,406.0",0.0,2,R1,0,01/01/2019
7,2,MORRIS PARK/VAN NEST,01 ONE FAMILY DWELLINGS,1,4090,37,nan,A1,1193 SACKET AVENUE,NaN,"10,461.0",1.0,0.0,1.0,"3,404","1,328.0","1,925.0",1,A1,0,01/01/2019
8,2,MORRIS PARK/VAN NEST,01 ONE FAMILY DWELLINGS,1,4120,18,nan,A5,1215 VAN NEST AVENUE,NaN,"10,461.0",1.0,0.0,1.0,"2,042","1,728.0","1,935.0",1,A5,0,01/01/2019


In [282]:
# Look at property sales for > 5,000 square feet
# Q. What is the highest square footage you see?
large_prop = df[df['gross_square_feet'] > 5000]
large_prop['gross_square_feet'].describe()

count             1,220.0
mean    41,495.5893442623
std     88,058.1593109996
min               5,004.0
25%               6,648.0
50%              11,352.0
75%              34,745.0
max           1,303,935.0
Name: gross_square_feet, dtype: float64

In [283]:
large_prop.head()

,borough,neighborhood,building_class_category,tax_class_at_present,block,lot,ease-ment,building_class_at_present,address,apartment_number,zip_code,residential_units,commercial_units,total_units,land_square_feet,gross_square_feet,year_built,tax_class_at_time_of_sale,building_class_at_time_of_sale,sale_price,sale_date
1,1,FASHION,21 OFFICE BUILDINGS,4,812,68,nan,O5,144 WEST 37TH STREET,NaN,"10,018.0",0.0,6.0,6.0,"2,962","15,435.0","1,920.0",4,O5,0,01/01/2019
2,1,FASHION,21 OFFICE BUILDINGS,4,839,69,nan,O5,40 WEST 38TH STREET,NaN,"10,018.0",0.0,7.0,7.0,"2,074","11,332.0","1,930.0",4,O5,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,nan,R1,"20 EAST 65TH STREET, B",B,"10,065.0",1.0,0.0,1.0,0,"6,406.0",0.0,2,R1,0,01/01/2019
5,1,UPPER EAST SIDE (79-96),07 RENTALS - WALKUP APARTMENTS,2B,1551,131,nan,C4,354 EAST 89TH STREET,NaN,"10,128.0",10.0,0.0,10.0,"2,013","6,570.0","1,920.0",2,C4,0,01/01/2019
6,1,UPPER WEST SIDE (96-116),07 RENTALS - WALKUP APARTMENTS,2B,1891,159,nan,C4,304 WEST 106 STREET,NaN,"10,025.0",10.0,0.0,10.0,"1,716","5,810.0","1,900.0",2,C4,0,01/01/2019


In [284]:
large_prop[['gross_square_feet', 'sale_price']].sort_values(by='gross_square_feet', ascending=False)[:10]

,gross_square_feet,sale_price
17,"1,303,935.0",0
26,"959,880.0",0
991,"805,467.0",565754371
7745,"697,029.0",850000000
14612,"662,532.0",0
986,"574,675.0",422000000
3998,"536,719.0",389858000
19744,"533,600.0",0
1267,"517,158.0",277700000
13846,"483,148.0",335000000


In [285]:
# What are the building class categories?
# How frequently does each occur?
df['building_class_category'].value_counts()

01 ONE FAMILY DWELLINGS                       5061
02 TWO FAMILY DWELLINGS                       4567
10 COOPS - ELEVATOR APARTMENTS                3471
13 CONDOS - ELEVATOR APARTMENTS               3339
03 THREE FAMILY DWELLINGS                     1438
07 RENTALS - WALKUP APARTMENTS                 807
09 COOPS - WALKUP APARTMENTS                   672
15 CONDOS - 2-10 UNIT RESIDENTIAL              421
04 TAX CLASS 1 CONDOS                          418
44 CONDO PARKING                               366
17 CONDO COOPS                                 300
05 TAX CLASS 1 VACANT LAND                     288
22 STORE BUILDINGS                             288
12 CONDOS - WALKUP APARTMENTS                  256
14 RENTALS - 4-10 UNIT                         200
29 COMMERCIAL GARAGES                          147
08 RENTALS - ELEVATOR APARTMENTS               120
30 WAREHOUSES                                  105
21 OFFICE BUILDINGS                             96
43 CONDO OFFICE BUILDINGS      

In [0]:
# Keep subset of rows:
# Sale price more than $0, 
# Building class category = Condos - Elevator Apartments
condos = df[(df['sale_price'] > 0) & (df['building_class_category'].str.contains('CONDO'))]
# Check how many rows you have now. (Should be 106 rows.)


In [287]:
condos.head()

,borough,neighborhood,building_class_category,tax_class_at_present,block,lot,ease-ment,building_class_at_present,address,apartment_number,zip_code,residential_units,commercial_units,total_units,land_square_feet,gross_square_feet,year_built,tax_class_at_time_of_sale,building_class_at_time_of_sale,sale_price,sale_date
65,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,691,1130,nan,R4,"505 WEST 19TH STREET, 7D",7D,"10,011.0",1.0,0.0,1.0,"18,975","1,077.0","2,013.0",2,R4,2000000,01/02/2019
70,1,UPPER WEST SIDE (59-79),13 CONDOS - ELEVATOR APARTMENTS,2,1167,1272,nan,R4,"235 WEST 75 STREET, 803",8S,"10,023.0",1.0,0.0,1.0,"28,335",545.0,"1,901.0",2,R4,2997613,01/02/2019
71,1,UPPER WEST SIDE (59-79),13 CONDOS - ELEVATOR APARTMENTS,2,1171,2200,nan,R4,"240 RIVERSIDE BOULEVARD, 4 F",4 F,"10,069.0",1.0,0.0,1.0,0,827.0,"2,004.0",2,R4,1250000,01/02/2019
72,1,UPPER WEST SIDE (79-96),13 CONDOS - ELEVATOR APARTMENTS,2,1213,1161,nan,R4,"127 WEST 82ND STREET, 3B",3B,"10,024.0",1.0,0.0,1.0,0,391.0,"1,912.0",2,R4,1955000,01/02/2019
73,1,UPPER WEST SIDE (79-96),13 CONDOS - ELEVATOR APARTMENTS,2,1213,1185,nan,R4,"127 WEST 82ND STREET, 2E",2E,"10,024.0",1.0,0.0,1.0,0,453.0,"1,912.0",2,R4,1955000,01/02/2019


In [289]:
# Make a Plotly Express scatter plot of GROSS_SQUARE_FEET vs SALE_PRICE
import plotly.express as px 
px.scatter(condos, 'gross_square_feet', 'sale_price')

In [290]:
# Add an OLS (Ordinary Least Squares) trendline,
# to see how the outliers influence the "line of best fit"
px.scatter(condos, 'gross_square_feet', 'sale_price', trendline='ols')

In [0]:
# Look at sales for more than $35 million

# All are at 70 Vestry Street
# All but one have the same SALE_PRICE & SALE_DATE
# Was the SALE_PRICE for each? Or in total?
# Is this dirty data?
over_35m = condos[condos['sale_price'] > 35000000]

In [293]:
over_35m[['sale_price', 'sale_date', 'gross_square_feet']].sort_values(by='sale_price', ascending=False)

,sale_price,sale_date,gross_square_feet
3995,239958219,01/23/2019,"23,029.0"
19202,200000000,04/05/2019,"286,755.0"
3698,189109922,01/22/2019,"11,905.0"
3699,189109922,01/22/2019,"162,971.0"
14395,75000000,03/13/2019,"8,102.0"
14394,75000000,03/13/2019,"8,102.0"
14393,75000000,03/13/2019,0.0
14392,75000000,03/13/2019,0.0
14391,75000000,03/13/2019,"90,307.0"
4779,64250000,01/25/2019,"9,138.0"


In [0]:
# Make a judgment call:
# Keep rows where sale price was < $35 million

# Check how many rows you have now. (Should be down to 90 rows.)


In [0]:
# Now that you've removed outliers,
# Look again at a scatter plot with OLS (Ordinary Least Squares) trendline


In [0]:
# Select these columns, then write to a csv file named tribeca.csv. Don't include the index.
